# ASSIGNMENT 3 - DEEP RL

## PART 2

### IMPORTING LIBRARIES

In [5]:
from sb3_contrib import MaskablePPO
from sb3_contrib.common.maskable.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from knapsack_env import BoundedKnapsackEnv
import numpy as np
import gymnasium as gym
import torch as th
from stable_baselines3.common.env_util import make_vec_env


In [6]:
#Defining the action mask for Maskable PPO

def mask_fn(env: gym.Env) -> np.ndarray:
    return env.get_mask()
def make_env():
    env = BoundedKnapsackEnv(n_items=200, max_weight=200, mask=True)

    # Wrapping up the environment on mask function
    env = ActionMasker(env, mask_fn)

    return env

vec_env = make_vec_env(make_env, n_envs=1)

# # Wrapping the environment with DummyVecEnv
# env = DummyVecEnv([lambda: env])

# # # Normalizing the observations and rewards
env = VecNormalize(vec_env, norm_obs=True, norm_reward= True, clip_obs= float('inf')) #Clipping done to avoid the error on missing argument

In [7]:
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=dict(pi=[128, 128], vf=[128, 128]))

model = MaskablePPO(MaskableActorCriticPolicy, env, n_epochs = 15, learning_rate=0.001, batch_size = 32, gamma = 0.99, policy_kwargs= policy_kwargs,  verbose=1, tensorboard_log="./ppo_knapsack_tensorboard/maskable")
model.learn(total_timesteps= 30000)

Using cuda device
Logging to ./ppo_knapsack_tensorboard/maskable\PPO_2


d:\Programs\minconda3\envs\syscon\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.action_masks to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search the reminding wrappers.
  logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 7.74     |
|    ep_rew_mean     | 374      |
| time/              |          |
|    fps             | 338      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 8.36      |
|    ep_rew_mean          | 428       |
| time/                   |           |
|    fps                  | 222       |
|    iterations           | 2         |
|    time_elapsed         | 18        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.2044988 |
|    clip_fraction        | 0.787     |
|    clip_range           | 0.2       |
|    entropy_loss         | -3.73     |
|    explained_variance   | -0.297    |
|    learning_rate        | 0.001     |
|    loss           

In [9]:
env.training = False # Setting env to unnormalized state
env.norm_reward = False

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, deterministic= True)
print(f"Mean reward: {mean_reward} ± {std_reward}")

Mean reward: 2404.0 ± 0.0
